# Games data

This file takes the player_seasons data and the player_matches_data, aggregates them, and creates a new dataframe in a form that can be easily input to a model

In [1]:
import pandas as pd
import numpy as np

In [2]:
NUM_PLAYERS_CONSIDERED_PER_TEAM = 7
ROLLING_GAMES_WINDOW = 5 # must match player_matches_data (should clean this up, non-dup)
PLAYER_MATCHES_FILENAME = "../data/20100101_20191118_player_matches.csv"
PLAYER_SEASONS_FILENAME = "../data/2009_2019_player_seasons_unique.csv"

In [3]:
player_matches = pd.read_csv(PLAYER_MATCHES_FILENAME)
player_seasons = pd.read_csv(PLAYER_SEASONS_FILENAME)

In [4]:
recent_stats_suffix = "_l" + str(ROLLING_GAMES_WINDOW)
player_matches = player_matches.sort_values(by=['game_id', 'fp' + recent_stats_suffix], ascending=[True, False])

team_game_group = player_matches.groupby(by=['game_id', 'home_game'])
player_matches = team_game_group.head(NUM_PLAYERS_CONSIDERED_PER_TEAM).reset_index(drop=True)

In [5]:
stats = ['fgm', 'fga', '3pm', '3pa', 'ftm', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tvr', 'pf', 'fp']
season_stats = ['player_id'] + [s + "_seas_avg" for s in stats]
recent_stats = [s + recent_stats_suffix for s in stats]

In [6]:
merge_season_fields = ['end_year'] + season_stats
to_merge_season_data = player_seasons[merge_season_fields]

In [7]:
merged_data = pd.merge(
    player_matches,
    to_merge_season_data,
    how='left',
    left_on=['player_id', 'season_start_year'],
    right_on = ['player_id', 'end_year']
).fillna(0)

In [8]:
y = ['fp']

all_data =  season_stats + recent_stats + y

In [9]:
per_game_df = pd.DataFrame()
game_group = merged_data.groupby('game_id')

for field in all_data:
    v = game_group.apply(lambda x: x[field].tolist())
    df = pd.DataFrame(v.tolist(), index=v.index).rename(columns=lambda x: x + 1).add_prefix(field + "_p").reset_index(drop=True)
    per_game_df = pd.concat([per_game_df, df], axis=1, sort=False)

In [10]:
per_game_df = per_game_df.fillna(0)

In [11]:
per_game_df

,player_id_p1,player_id_p2,player_id_p3,player_id_p4,player_id_p5,player_id_p6,player_id_p7,player_id_p8,player_id_p9,player_id_p10,...,fp_p5,fp_p6,fp_p7,fp_p8,fp_p9,fp_p10,fp_p11,fp_p12,fp_p13,fp_p14
0,robinna01,horfoal01,johnsjo02,chandwi01,smithjo03,bibbymi01,gallida01,harrial01,leeda02,duhonch01,...,49.5,31.9,30.7,27.8,25.4,24.4,16.9,14.2,12.2,11.6
1,bryanko01,hawessp01,udrihbe01,gasolpa01,casspom01,odomla01,bynuman01,udokaim01,fishede01,rodrise01,...,43.7,43.5,34.3,25.5,19.1,15.5,10.2,10.1,9.9,7.9
2,flynnjo01,howardw01,anderry01,nelsoja01,lewisra02,barnema02,cartevi01,jeffeal01,loveke01,ellinwa01,...,37.9,34.7,34.4,33.5,30.5,23.3,21.1,18.2,16.7,14.2
3,wallara01,boshch01,allenra02,allento01,turkohe01,jackja01,bargnan01,perkike01,houseed01,davisgl01,...,35.9,35.9,34.3,33.5,29.8,24.0,14.5,14.1,12.4,11.2
4,howardw01,anderry01,nelsoja01,lewisra02,barnema02,cartevi01,rosede01,denglu01,gibsota01,salmojo01,...,45.3,20.6,47.7,39.8,36.4,27.8,22.8,22.7,20.1,19.5
5,flynnjo01,jeffeal01,loveke01,ellinwa01,wilkida02,hibbero01,headlu01,watsoea01,mcrobjo01,dunlemi02,...,15.9,47.4,39.8,32.3,30.6,30.1,29.1,27.9,20.3,17.8
6,wadedw01,jacksst02,richaqu01,wallage01,hasleud01,diawbo01,augusdj01,feltora01,beaslmi01,onealje01,...,28.0,25.5,24.7,24.6,24.2,24.0,21.3,18.2,13.9,11.4
7,westbru01,duranke01,bogutan01,reddmi01,maynoer01,ilyaser01,jennibr01,warriha01,sefolth01,ridnolu01,...,32.2,32.0,28.9,26.6,22.8,22.2,20.9,19.4,19.4,18.6
8,jamesle01,harride01,lopezbr01,jianlyi01,varejan01,onealsh01,douglch01,willima01,parkean01,leeco01,...,35.4,27.3,27.1,26.9,18.8,17.7,14.2,13.8,9.2,7.4
9,paulch01,arizatr01,okafoem01,hayesch01,lowryky01,brookaa01,westda01,battish01,brownde02,scolalu01,...,30.2,27.2,27.0,23.8,18.9,18.6,17.2,17.1,13.7,9.7


In [12]:
path = "../data/games_" + str(NUM_PLAYERS_CONSIDERED_PER_TEAM) + "_players" + ".csv"
per_game_df.to_csv(path, index = False)

## Duplicate the rows (for each game, each player gets to be player_1)

In [13]:
def swap_colums_orders(colnames, i, j):
    pi = "_p" + str(i)
    pj = "_p" + str(j)
    swapped_colnames = []
    for col in colnames :
        new_col = col
        if col.find(pi, len(col)-len(pi)) > -1:
            new_col = col.replace(pi, pj)
        if col.find(pj, len(col)-len(pj)) > -1:
            new_col = col.replace(pj, pi)
        swapped_colnames.append(new_col)
    return swapped_colnames
def swap_team_columns(colnames):
    swaped_teams = colnames
    for i in range(1,8):
        swaped_teams = swap_colums_orders(swaped_teams, i, 7+i)
    return swaped_teams

In [15]:
colnames_all = per_game_df.columns
all_per_game_df = per_game_df

for team in range(2) :
    for i in range(2+7*team, 8+7*team):
        temps_df = per_game_df[swap_colums_orders(colnames_all, 1+7*team, i)]
        if team==1:
            temps_df = temps_df[swap_team_columns(colnames_all)]
        temps_df.columns = colnames_all
        all_per_game_df = pd.concat([all_per_game_df, temps_df])
all_per_game_df.drop(["fp_p"+str(i) for i in range(2,15)], axis=1)

,player_id_p1,player_id_p2,player_id_p3,player_id_p4,player_id_p5,player_id_p6,player_id_p7,player_id_p8,player_id_p9,player_id_p10,...,fp_l5_p6,fp_l5_p7,fp_l5_p8,fp_l5_p9,fp_l5_p10,fp_l5_p11,fp_l5_p12,fp_l5_p13,fp_l5_p14,fp_p1
0,robinna01,horfoal01,johnsjo02,chandwi01,smithjo03,bibbymi01,gallida01,harrial01,leeda02,duhonch01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,69.2
1,bryanko01,hawessp01,udrihbe01,gasolpa01,casspom01,odomla01,bynuman01,udokaim01,fishede01,rodrise01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,65.5
2,flynnjo01,howardw01,anderry01,nelsoja01,lewisra02,barnema02,cartevi01,jeffeal01,loveke01,ellinwa01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,51.9
3,wallara01,boshch01,allenra02,allento01,turkohe01,jackja01,bargnan01,perkike01,houseed01,davisgl01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,45.2
4,howardw01,anderry01,nelsoja01,lewisra02,barnema02,cartevi01,rosede01,denglu01,gibsota01,salmojo01,...,34.400,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,30.9
5,flynnjo01,jeffeal01,loveke01,ellinwa01,wilkida02,hibbero01,headlu01,watsoea01,mcrobjo01,dunlemi02,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24.1
6,wadedw01,jacksst02,richaqu01,wallage01,hasleud01,diawbo01,augusdj01,feltora01,beaslmi01,onealje01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,66.1
7,westbru01,duranke01,bogutan01,reddmi01,maynoer01,ilyaser01,jennibr01,warriha01,sefolth01,ridnolu01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,56.3
8,jamesle01,harride01,lopezbr01,jianlyi01,varejan01,onealsh01,douglch01,willima01,parkean01,leeco01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,60.3
9,paulch01,arizatr01,okafoem01,hayesch01,lowryky01,brookaa01,westda01,battish01,brownde02,scolalu01,...,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,54.9


In [18]:
path = "../data/all_games_" + str(NUM_PLAYERS_CONSIDERED_PER_TEAM) + "_players" + ".csv"
all_per_game_df.to_csv(path, index = False)